In [1]:
import pandas as pd
import os
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

from utils.param_objects import EmpatheticDialogueParams, MELDParams
from models.emotimebert import EmotionalTimeBert
from sklearn.metrics import f1_score
from tqdm import tqdm
import matplotlib.pyplot as plt
from data_handling.datasets.empathetic_dialogues import EmpatheticDialoguesDataset
from data_handling.data_cleaning.empathetic_dialogues import load_empath_conversations, load_empath_test_conversations
from data_handling.data_cleaning.meld import load_meld_dfs, load_meld_split
from utils.utils import train_model, validate_model, test_model, collate_conversations, create_new_run_dir, save_table_as_csv

from pathlib import Path
import re
os.environ["TOKENIZERS_PARALLELISM"] = "false"
TRAINING = False
''

''

In [2]:
base_save_location = create_new_run_dir(new_dir=TRAINING)

In [4]:
conversations, val_conversations, emotion_labels, emotion_to_id = load_empath_conversations()

meld_data = load_meld_split("train")
meld_data

,serial_number,utterance,speaker,emotion,dialogue_id,utterance_id,season,episode,start_time,end_time
0,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]",[also I was the point person on my companys t...,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]","[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[976059, 981940, 983442, 986820, 994452, 10011...","[981731, 983442, 986389, 989572, 1000917, 1004..."
1,"[15, 16, 17, 18, 19, 20, 23]",[But then who? The waitress I went out with la...,"[0, 1, 0, 1, 0, 0, 1]","[3, 4, 3, 2, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1]","[0, 1, 2, 3, 4, 5, 8]","[9, 9, 9, 9, 9, 9, 9]","[23, 23, 23, 23, 23, 23, 23]","[2200364, 2204368, 2204368, 2209290, 2212376, ...","[2202824, 2206578, 2209122, 2211791, 2213543, ..."
2,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[Hey, Mon., Hey-hey-hey. You wanna hear someth...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]","[0, 0, 5, 4, 3, 0, 6, 4, 0, 0, 5, 0, 5]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]","[350350, 352435, 353897, 357023, 358734, 35994...","[351433, 353897, 355743, 358734, 359774, 36215..."
3,"[37, 38, 39, 40, 41, 42, 43, 45, 46]","[Oh my God, oh my God! Poor Monica!, What, wha...","[0, 1, 0, 0, 0, 0, 0, 0, 2]","[3, 3, 3, 0, 0, 0, 3, 4, 0]","[3, 3, 3, 3, 3, 3, 3, 3, 3]","[0, 1, 2, 3, 4, 5, 6, 8, 9]","[3, 3, 3, 3, 3, 3, 3, 3, 3]","[12, 12, 12, 12, 12, 12, 12, 12, 12]","[616115, 618618, 619750, 621078, 623664, 62666...","[618450, 619750, 620966, 623496, 626499, 62958..."
4,"[47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 5...","[Hey!, Hi!, What are you doing here?, Ah y'kno...","[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]","[1, 5, 1, 0, 0, 0, 0, 0, 5, 5, 5, 4, 0, 0, 0]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...","[1223931, 1224891, 1225140, 1226433, 1229394, ...","[1224891, 1225028, 1226265, 1229185, 1230771, ..."
...,...,...,...,...,...,...,...,...,...,...
1033,"[10427, 10428, 10429]","[Hey, Mon, I was just doing the dishes!, Hey!,...","[0, 1, 0]","[0, 5, 1]","[1034, 1034, 1034]","[0, 1, 2]","[4, 4, 4]","[20, 20, 20]","[259759, 261010, 264597]","[263595, 263595, 266390]"
1034,"[10430, 10431, 10432, 10433, 10434, 10435, 10436]","[What about me?! You-you just said I could!, I...","[0, 1, 0, 2, 1, 2, 0]","[7, 0, 7, 0, 0, 7, 5]","[1035, 1035, 1035, 1035, 1035, 1035, 1035]","[0, 1, 2, 3, 4, 5, 6]","[4, 4, 4, 4, 4, 4, 4]","[22, 22, 22, 22, 22, 22, 22]","[158074, 160034, 165206, 167208, 167930, 17225...","[159867, 165038, 167040, 167930, 169854, 17517..."
1035,"[10437, 10438, 10439, 10440, 10441, 10442, 104...",[Hey-hey! Stanley! Hey-hey! You're leading man...,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, ...","[5, 4, 3, 4, 1, 0, 3, 0, 4, 0, 7, 0, 0, 0, 0, ...","[1036, 1036, 1036, 1036, 1036, 1036, 1036, 103...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 2...","[1154987, 1161201, 1166331, 1167666, 1172296, ...","[1161033, 1165371, 1167478, 1170001, 1174348, ..."
1036,"[10458, 10459, 10460]","[Rachel, do you have any muffins left?, Yeah, ...","[0, 1, 0]","[0, 0, 0]","[1037, 1037, 1037]","[0, 1, 2]","[2, 2, 2]","[2, 2, 2]","[164706, 166098, 167792]","[166098, 167672, 172963]"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
dataset = EmpatheticDialoguesDataset(conversations, tokenizer)
val_dataset = EmpatheticDialoguesDataset(val_conversations, tokenizer)
loader = DataLoader(
    dataset,
    batch_size=40,
    shuffle=True,
    num_workers=8,
    # pin_memory=True,
    # persistent_workers=True,
    collate_fn=lambda x: collate_conversations(x, tokenizer),
)

val_loader = DataLoader(
    val_dataset,
    batch_size=40,
    shuffle=False, # future me, keep it false, helps reproduce results
    num_workers=8,
    #pin_memory=True,
    # persistent_workers=True,
    collate_fn=lambda x: collate_conversations(x, tokenizer),
)


In [ ]:
device = "cpu"

if torch.cuda.is_available():
    device = "cuda"
    print("Device:", torch.cuda.get_device_name(0))
    print("CUDA Enabled!")

EDParams = EmpatheticDialogueParams(num_labels=len(emotion_labels))

model = EmotionalTimeBert("./medbert_4_epochs", EDParams).to(device)

if not TRAINING:
    model.load_state_dict(torch.load(f"{base_save_location}/model.pt")) #  emotional_time_bert_GRU_To_Show5.pt"

criterion = nn.CrossEntropyLoss(ignore_index=-1)

optimizer = torch.optim.AdamW([
    {"params": model.encoder.encoder.layer[-2:].parameters(), "lr": 1e-5},
    {"params": model.temporal_transformer.parameters(), "lr": 3e-4},
    {"params": model.time_embed.parameters(), "lr": 3e-4},
    {"params": model.speakers_embed.parameters(), "lr": 3e-4},
    {"params": model.head_emotions.parameters(), "lr": 3e-4},
])

In [ ]:
train_step_losses = []
val_step_losses = []
global_steps = []
val_steps = []


if TRAINING:
    num_of_epochs = 5
    step = 0
    for epoch in range(num_of_epochs):
        progress_bar = tqdm(loader, total=len(loader))
        avg_loss, step = train_model(model, optimizer, device, criterion=criterion, bar=progress_bar, train_step_losses=train_step_losses, global_steps=global_steps, start_step=step)
        validate_progress = tqdm(val_loader, total=len(val_loader))
        val_loss, val_f1, step = validate_model(model, device, criterion=criterion, bar=validate_progress, val_step_losses=val_step_losses, val_steps=val_steps, start_step=step)

        print(f"Epoch {epoch+1}: train loss = {avg_loss:.4f}")
        print(f"Epoch {epoch+1}: val loss   = {val_loss:.4f}")
        print(f"Epoch {epoch+1}: val F1     = {val_f1:.4f}")

    torch.save(model.state_dict(), f"{base_save_location}/model.pt")
    save_table_as_csv(global_steps, f"{base_save_location}/global_steps.csv")
    save_table_as_csv(train_step_losses, f"{base_save_location}/train_step_losses.csv")
    save_table_as_csv(val_step_losses, f"{base_save_location}/val_step_losses.csv")
    save_table_as_csv(val_steps, f"{base_save_location}/val_steps.csv")



In [ ]:
if TRAINING:
    plt.figure(figsize=(10, 5))
    plt.plot(global_steps, train_step_losses, alpha=0.4, label="Train (per batch)")
    plt.plot(val_steps, val_step_losses, alpha=0.8, label="Val (per batch)")

    plt.xlabel("Training Step")
    plt.ylabel("Loss")
    plt.title("Per-batch Training & Validation Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{base_save_location}/Unsmoothed.png")

In [ ]:
def ema(values, beta=0.98):
    smoothed = []
    avg = values[0]
    for v in values:
        avg = beta * avg + (1 - beta) * v
        smoothed.append(avg)
    return smoothed

if TRAINING:
    plt.figure(figsize=(10, 5))
    plt.plot(global_steps, ema(train_step_losses), label="Train")
    plt.plot(val_steps, ema(val_step_losses), label="Val")
    plt.xlabel("Training Step")
    plt.ylabel("Loss")
    plt.title("Per-step Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{base_save_location}/Smoothed.png")

In [ ]:
test_conversations = load_empath_test_conversations(emotion_to_id)

In [ ]:
test_dataset = EmpatheticDialoguesDataset(test_conversations, tokenizer)
test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=8,
    collate_fn=lambda x: collate_conversations(x, tokenizer)
)

In [ ]:
test_f1 = test_model(
    model=model,
    dataloader=test_loader,
    device=device,
    emotion_labels=emotion_labels,
    save_results=True,
    save_dir=f"{base_save_location}/classification_report.csv",
)
# also look into bert's special keywords

In [ ]:
def ablation(model, dataloader, device, ablate):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            utterance_mask = batch["utterance_mask"].to(device)

            timestamps = batch["timestamps"].to(device)
            speakers = batch["speakers"].to(device)

            if ablate == "time_zero":
                timestamps = torch.zeros_like(timestamps)
            elif ablate == "time_shuffle":
                timestamps = timestamps[torch.randperm(timestamps.size(0))]
            elif ablate == "speaker_zero":
                speakers = torch.zeros_like(speakers)

            logits = model(
                input_ids, attention_mask,
                timestamps, speakers,
                labels, utterance_mask
            )

            preds = logits.argmax(-1)
            mask = labels != -1

            all_preds.append(preds[mask].cpu())
            all_labels.append(labels[mask].cpu())

    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)

    return f1_score(all_labels, all_preds, average="macro")

In [ ]:
print("Normal:", ablation(model, val_loader, device, "none"))

In [ ]:
print("Time zero:", ablation(model, val_loader, device, "time_zero"))

In [ ]:
print("Time shuffle:", ablation(model, val_loader, device, "time_shuffle"))

In [ ]:
print("Speaker zero:", ablation(model, val_loader, device, "speaker_zero"))